In [1]:
# Obtiene los datos de precipitación par la Cuenca del Valle
# de Méxco a partir de CHIRPS y los promedia espacialmente.

import pandas as pd
import numpy as np

import xarray as xr

from scipy.stats import linregress

# Datos
path_d = "../results/onset/"
path_r = "../results/onset/graficas/"

# Variable, nivel de presión y región.
vars = [ "u", "v", "gp", "sst", "olr", "sp", #"vidmf",
    "vivfu", "vivfv" ]
levels = [ "925", "200" ]
region = [ "mexico" ]

# Cantidad de días promediados.
d = 20

# Fechas de onset por precipitación
date_pre = pd.read_csv(path_d + "onset_CHIRPS.csv", index_col = "Año" )
date_pre.index = pd.RangeIndex(date_pre.index[0], date_pre.index[-1] + 1, 
    name = "Año" )
date_pre.iloc[:, 0] = pd.to_datetime( date_pre.iloc[:, 0] )
date_pre.iloc[:, 1] = pd.to_datetime( date_pre.iloc[:, 1] )

# Se convierten las fechas a días julianos.
date_pre["Cuenca - Onset"] = date_pre["Cuenca - Onset"].apply(
    lambda x: pd.to_datetime(x) )
date_pre["Cuenca - Onset"] = date_pre["Cuenca - Onset"].apply(
    lambda x: x.dayofyear )

df = pd.read_csv(path_d + "pre_daily_CHIRPS.csv", index_col = "time" )
df.index = pd.to_datetime( df.index )

In [ ]:
# Crea un índice de onset de acuerdo con
# el umbral para n días seguidos de
# cumplimiento con el criterio.
def pre_onset( umbral = [2.25, 2.25], n = [10, 10], m_i = 4 ):

    df_mean_ind = pd.DataFrame( df.copy(), columns = ["Cuenca"] )

    years = range(1981, 2021)
    c = "Cuenca"
    title = "onset_withdrawal_CHIRPS_diferente.png"

    # Onset.
    # Nombre de las columnas dummy.
    proc = [c]
    for i in range(0, n[0]):
        proc.append(c + "_" + str(i)) 
    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    for i, p in enumerate(proc[1:]):
        df_mean_ind[p] = df_mean_ind[proc[0]].shift(i)
        df_mean_ind[p].clip( upper = umbral[0],
            inplace = True )
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] == umbral[0], 0)
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] < umbral[0], 1)
    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df_mean_ind[ind] = 0
    for i in range(n[0]):
        df_mean_ind[ind] += df_mean_ind[proc[i + 1]]
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] == n[0], 0 )
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] < n[0], 1 )
    df_mean_ind[ind] = df_mean_ind[ind].where(
        df_mean_ind.index.month.isin( range(5, 12) ), 0 )
    df_mean_ind.drop( proc[1:], axis = 1, inplace = True )
    # Crea una tabla con los valores anuales.
    col = []
    name = "Cuenca"
    col.append(name + " - Onset")
    col.append(name + " - Withdrawal")
    date = pd.DataFrame()
    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df_mean_ind[ (df_mean_ind.index.year == j)
            & df_mean_ind.index.month.isin(range(m_i, 12)) ]
        row = []
        # Fechas.
        row.append( df_year.iloc[:, 1].idxmax() )
        row.append( None )
        #row.append( df_year.iloc[::-1, 1].idxmax() )
        # Agregamos la fila al DataFrame.
        date = date.append([row])
    # Recorremos al inicio de la racha de días de precipitación.
    date.iloc[:, 0] = date.iloc[:, 0] - np.timedelta64(n[0], 'D')

    # Withdrawal.
    # Nombre de las columnas dummy.
    proc = [c]
    for i in range(0, n[1]):
        proc.append(c + "_" + str(i)) 
    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    for i, p in enumerate(proc[1:]):
        df_mean_ind[p] = df_mean_ind[proc[0]].shift(i)
        df_mean_ind[p].clip( upper = umbral[1],
            inplace = True )
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] == umbral[1], 0)
        df_mean_ind[p] = df_mean_ind[p].where(
            df_mean_ind[p] < umbral[1], 1)
    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df_mean_ind[ind] = 0
    for i in range(n[1]):
        df_mean_ind[ind] += df_mean_ind[proc[i + 1]]
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] == n[1], 0 )
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind[ind] < n[1], 1 )
    df_mean_ind[ind] = df_mean_ind[ind].where(
        df_mean_ind.index.month.isin( range(5, 12) ), 0 )
    df_mean_ind.drop( proc[1:], axis = 1, inplace = True )
    # Crea una tabla con los valores anuales.
    col = []
    name = "Cuenca"
    col.append(name + " - Onset")
    col.append(name + " - Withdrawal")
    date_2 = pd.DataFrame()
    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df_mean_ind[ (df_mean_ind.index.year == j)
            & df_mean_ind.index.month.isin(range(m_i, 12)) ]
        row = []
        # Fechas.
        row.append( None )
        #row.append( df_year.iloc[:, 1].idxmax() )
        row.append( df_year.iloc[::-1, 1].idxmax() )
        # Agregamos la fila al DataFrame.
        date_2 = date_2.append([row])
    # Recorremos al inicio de la racha de días de precipitación.
    date.iloc[:, 1] = date.iloc[:, 1] - np.timedelta64(n[1], 'D')

    # Unimos onset y withdrawal.
    date.iloc[:, 1] = date_2.iloc[:, 1]

    # Damos formato a las columnas y al índice.
    date.columns = col
    date.index = years
    date.index.name = "Año"

    # Longitud de la temporada.
    date[name + " - Length"] = date.iloc[:, 1] - date.iloc[:, 0]

    # Calculamos el shift necesario para alinear las fechas.
    shift = date.iloc[:, 0].apply(lambda x: x.dayofyear)
    shift_2 = date.iloc[:, 0].apply(lambda x: x.dayofyear)

    for i in [0, 1]:
        date.iloc[:, i] = date.iloc[:, i].apply( 
            lambda x: pd.to_datetime(x) )
        date.iloc[:, i] = date.iloc[:, i].apply(
            lambda x: x.dayofyear )

    return date, shift, shift_2

def var_onset( date_pre, df, onset = True, vars = [-12.5, 7] , m_i = 5 ):

    umbral = vars[0]
    n = int( vars[1] )

    c = "Cuenca"

    # Nombre de las columnas dummy.
    proc = [c]
    for i in range( 0, n ): proc.append(c + "_" + str(i))

    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    # Umbral positivo.
    if umbral >= 0:
        for i, p in enumerate(proc[1:]):
            df[p] = df[proc[0]].shift(i)
            df[p].clip( upper = umbral, inplace = True )
            df[p] = df[p].where( df[p] == umbral, 0 )
            df[p] = df[p].where( df[p] < umbral, 1 )
    # Umbral negativo
    if umbral < 0:
        for i, p in enumerate(proc[1:]):
            df[p] = df[proc[0]].shift(i)
            df[p].clip( lower = umbral, inplace = True )
            df[p] = df[p].where( df[p] == umbral, 0 )
            df[p] = df[p].where( df[p] > umbral, 1 )

    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df[ind] = 0
    for i in range( n ):
        df[ind] += df[proc[i + 1]]
    df[ind] = df[ind].where( df[ind] == n, 0 )
    df[ind] = df[ind].where( df[ind] < n, 1 )
    df[ind] = df[ind].where( df.index.month.isin( range(m_i, 12) ), 0 )
    df.drop(proc[1:], axis = 1, inplace = True)

    # Crea una tabla con los valores anuales.
    date = date_pre["Cuenca - Onset"].copy()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df[ (df.index.year == j)
            & df.index.month.isin(range(m_i, 12)) ]

        # Fechas.
        date[j] = df_year.iloc[:, 1].idxmax()

    # Crea una tabla con los valores anuales.
    col = []
    name = "Cuenca"
    col.append(name + " - Onset")
    col.append(name + " - Withdrawal")

    date = pd.DataFrame()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y diciembre.
        df_year = df[ (df.index.year == j)
            & df.index.month.isin(range(m_i, 12)) ]

        row = []

        # Fechas.
        row.append( df_year.iloc[:, 1].idxmax() )
        row.append( df_year.iloc[::-1, 1].idxmax() )

        # Agregamos la fila al DataFrame.
        date = date.append([row])

    # Damos formato a las columnas y al índice.
    date.columns = col
    date.index = years
    date.index.name = "Año"
    # Recorremos al inicio de la racha de días de precipitación.
    date = date - np.timedelta64(n, 'D')

    # Correlación entre índice de precipitación y de viento.
    # Se convierten las fechas a días julianos.
    for i in [0, 1]:
        date.iloc[:, i] = date.iloc[:, i].apply( lambda x: pd.to_datetime(x) )
        date.iloc[:, i] = date.iloc[:, i].apply( lambda x: x.dayofyear )
        
    # Correlaciones de las series.
    if onset: i = 0
    else:     i = 1
    linreg = linregress(date.iloc[:, i], date_pre.iloc[:, i])
    corr = linreg.rvalue
    #corr = date.corr( date_pre["Cuenca - Onset"] )

    # Liberamos memoria.
    del date

    return corr

def ind_max( df, ds, onset = True, zone = (-80, -70, 12.5, 17.5),
    n = (1, 21), num = 10 ):
    # Precipitación diaria promedio en toda la cuenca.
    data = ( ds.sel( longitude = slice( zone[0], zone[1] ), 
        latitude = slice( zone[3], zone[2] ) )
        .mean(dim = ["latitude", "longitude"]).to_dataframe() )
    data.columns = ["Cuenca"]

    u = [ np.fix( data.min()[0] * 2 ) / 2,
            np.fix( data.max()[0] * 2 ) / 2, num ]

    if not np.isnan( data.min()[0] ):

        c = np.stack( np.meshgrid( np.linspace( *u ), np.arange( *n ) ),
            axis = 2 )
        res = np.ndarray(shape = c.shape[0:2])

        for i in range(c.shape[0]):
            for j in range(c.shape[1]):
                res[i, j] = var_onset( date_pre = df, df = data,
                    onset = onset, vars = c[i, j] )

        return ( res.max(), c[ np.argmax(res) // res.shape[1],
            np.argmax(res) % res.shape[1] ] )

    else: return ( 0, [0, 0] )

def eval(df, onset = True, anom = [True, False],
    v = 0, n = 0, step = 20, num = 10):

    # Variable a utilizar.
    if v > 2: lev = ""
    else: lev = "_" + levels[n]

    lon = np.arange(-120, -50 + step, step)
    lat = np.arange(0 , 40 + step, step)

    lon = np.stack( [ lon[0:-1], lon[1:] ], axis = 1)
    lat = np.stack( [ lat[0:-1], lat[1:] ], axis = 1)

    a = np.repeat(lon, lat.shape[0], axis = 0)
    b = np.tile(lat, [lon.shape[0], 1])

    bounds = np.concatenate([a, b], axis = 1)

    A = []
    if anom[0]: A.append(".")
    if anom[1]: A.append("_anom.")
    for a in A:
        x = []
        y = np.ndarray([0])

        fname = ( "onset_" + vars[v] + lev + "_mean_"
            + str(d) + "_dias" + a + "grib" )
        ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
        if v < 3:
            ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
        elif v < 6:
            ds = ds.drop_vars(["step", "surface", "valid_time"])
        else:
            ds = ds.drop_vars(["step", "entireAtmosphere", "valid_time"])

        for i in bounds:
            z = ind_max( df = df, ds = ds, onset = onset, zone = i, num = num )
            x.append( z )
            y = np.append( y, [z[0]], axis = 0 )

        print( ( f"var: {vars[v]}, lev: {levels[n]}, "
            + f"bounds: {bounds[np.argmax(y)]}, "
            + f"corr: {x[np.argmax(y)][0]:.3f}, "
            + f"umbral: {x[np.argmax(y)][1][0]:.3f}, " 
            + f"n. días: {int(x[np.argmax(y)][1][1])}" ) )

def eval_zona(df, onset = True, anom = [True, False],
    zone = (-80, -70, 12.5, 17.5), v = 0, n = 0, step = 20, num = 10):

    # Variable a utilizar.
    if v > 2: lev = ""
    else: lev = "_" + levels[n]

    A = []
    if anom[0]: A.append(".")
    if anom[1]: A.append("_anom.")
    for a in A:
        x = []
        y = np.ndarray([0])

        fname = ( "onset_" + vars[v] + lev + "_mean_"
            + str(d) + "_dias" + a + "grib" )
        ds = xr.open_dataset( path_d + fname, engine = "cfgrib" )
        if v < 3:
            ds = ds.drop_vars(["step", "isobaricInhPa", "valid_time"])
        elif v < 6:
            ds = ds.drop_vars(["step", "surface", "valid_time"])
        else:
            ds = ds.drop_vars(["step", "entireAtmosphere", "valid_time"])

        z = ind_max( df = df, ds = ds, onset = onset, zone = zone, num = num )

        print( ( f"var: {vars[v]}, lev: {levels[n]}, "
            + f"bounds: {zone}, "
            + f"corr: {z[0]:.3f}, "
            + f"umbral: {z[1][0]:.3f}, " 
            + f"n. días: {int(z[1][1])}" ) )

def check( onset = True, zona = False, anom = [True, False],
    U = [ 3 ], zone = (-80, -70, 12.5, 17.5), step = 20, num = 10 ):
    for u in U:
        print( f"\nUmbral: {u} mm/dia" )
        date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
        for v in range(0, 3):
            for n in range(0, 2):
                if zona:
                    eval_zona( df = date_pre, onset = onset, anom = anom,
                        zone = zone, v = v, n = n, step = step, num = num )
                else:
                    eval(df = date_pre, onset = onset, anom = anom,
                        v = v, n = n, step = step, num = num)
        for v in range(3, 8):
            if zona:
                eval_zona(df = date_pre, onset = onset, anom = anom,
                    zone = zone, v = v, step = step, num = num)
            else:
                eval(df = date_pre, onset = onset, anom = anom,
                    v = v, n = n, step = step, num = num)

In [5]:
check( U = [2.5] )


Umbral: 2.5 mm/dia
var: 0, lev: 0, bounds: [-100  -80    0   20], corr: 0.370, umbral: 1.250, n. días: 14
var: 0, lev: 1, bounds: [-100  -80    0   20], corr: 0.365, umbral: 5.500, n. días: 19
var: 1, lev: 0, bounds: [-60 -40  20  40], corr: 0.356, umbral: 2.875, n. días: 1
var: 1, lev: 1, bounds: [-100  -80   20   40], corr: 0.351, umbral: 3.250, n. días: 9
var: 2, lev: 0, bounds: [-100  -80   20   40], corr: 0.379, umbral: 8061.500, n. días: 1
var: 2, lev: 1, bounds: [-80 -60  20  40], corr: 0.263, umbral: 122179.000, n. días: 1
var: 3, lev: 0, bounds: [-80 -60   0  20], corr: 0.405, umbral: 302.500, n. días: 20
var: 4, lev: 0, bounds: [-120 -100    0   20], corr: 0.394, umbral: -275.500, n. días: 1
var: 5, lev: 0, bounds: [-100  -80   20   40], corr: 0.365, umbral: 99815.500, n. días: 3
var: 6, lev: 0, bounds: [-60 -40   0  20], corr: 0.309, umbral: -256.250, n. días: 19
var: 7, lev: 0, bounds: [-120 -100   20   40], corr: 0.362, umbral: 16.750, n. días: 4


In [6]:
check( onset = False, U = [2.5] )


Umbral: 2.5 mm/dia
var: 0, lev: 0, bounds: [-60 -40  20  40], corr: 0.468, umbral: 2.750, n. días: 12
var: 0, lev: 1, bounds: [-60 -40   0  20], corr: 0.399, umbral: 9.500, n. días: 3
var: 1, lev: 0, bounds: [-120 -100   20   40], corr: 0.382, umbral: 0.125, n. días: 6
var: 1, lev: 1, bounds: [-80 -60  20  40], corr: 0.324, umbral: 5.875, n. días: 2
var: 2, lev: 0, bounds: [-60 -40   0  20], corr: 0.341, umbral: 7725.625, n. días: 3
var: 2, lev: 1, bounds: [-120 -100    0   20], corr: 0.446, umbral: 121429.000, n. días: 10
var: 3, lev: 0, bounds: [-120 -100    0   20], corr: 0.475, umbral: 300.000, n. días: 18
var: 4, lev: 0, bounds: [-60 -40   0  20], corr: 0.363, umbral: -275.625, n. días: 4
var: 5, lev: 0, bounds: [-60 -40   0  20], corr: 0.412, umbral: 100599.625, n. días: 1
var: 6, lev: 0, bounds: [-60 -40   0  20], corr: 0.479, umbral: -256.250, n. días: 14
var: 7, lev: 0, bounds: [-120 -100   20   40], corr: 0.377, umbral: 16.750, n. días: 12


In [7]:
check()


Umbral: 3.5 mm/dia
var: 0, lev: 0, bounds: [-60 -40   0  20], corr: 0.529, umbral: -7.250, n. días: 10
var: 0, lev: 1, bounds: [-100  -80    0   20], corr: 0.547, umbral: 5.500, n. días: 19
var: 1, lev: 0, bounds: [-120 -100   20   40], corr: 0.440, umbral: -1.625, n. días: 6
var: 1, lev: 1, bounds: [-100  -80    0   20], corr: 0.435, umbral: 3.500, n. días: 5
var: 2, lev: 0, bounds: [-100  -80   20   40], corr: 0.391, umbral: 7762.000, n. días: 13
var: 2, lev: 1, bounds: [-60 -40   0  20], corr: 0.367, umbral: 122583.500, n. días: 1
var: 3, lev: 0, bounds: [-80 -60   0  20], corr: 0.492, umbral: 302.500, n. días: 2
var: 4, lev: 0, bounds: [-100  -80   20   40], corr: 0.357, umbral: -264.000, n. días: 20
var: 5, lev: 0, bounds: [-100  -80   20   40], corr: 0.339, umbral: 99815.500, n. días: 3
var: 6, lev: 0, bounds: [-80 -60  20  40], corr: 0.438, umbral: -134.500, n. días: 1
var: 7, lev: 0, bounds: [-80 -60   0  20], corr: 0.438, umbral: 93.000, n. días: 1


In [8]:
check(onset = False)


Umbral: 3.5 mm/dia
var: 0, lev: 0, bounds: [-60 -40  20  40], corr: 0.499, umbral: 2.750, n. días: 14
var: 0, lev: 1, bounds: [-120 -100    0   20], corr: 0.291, umbral: 8.000, n. días: 2
var: 1, lev: 0, bounds: [-80 -60  20  40], corr: 0.364, umbral: -2.750, n. días: 5
var: 1, lev: 1, bounds: [-100  -80   20   40], corr: 0.334, umbral: 3.250, n. días: 19
var: 2, lev: 0, bounds: [-120 -100    0   20], corr: 0.427, umbral: 7458.500, n. días: 1
var: 2, lev: 1, bounds: [-60 -40   0  20], corr: 0.285, umbral: 121179.500, n. días: 12
var: 3, lev: 0, bounds: [-120 -100   20   40], corr: 0.449, umbral: 300.000, n. días: 3
var: 4, lev: 0, bounds: [-100  -80    0   20], corr: 0.570, umbral: -247.750, n. días: 13
var: 5, lev: 0, bounds: [-120 -100   20   40], corr: 0.459, umbral: 91100.250, n. días: 12
var: 6, lev: 0, bounds: [-60 -40  20  40], corr: 0.452, umbral: 194.750, n. días: 5
var: 7, lev: 0, bounds: [-100  -80   20   40], corr: 0.452, umbral: -1.000, n. días: 7


In [9]:
for u in [ 2.5, 3, 3.5 ]:
    print( f"\nUmbral: {u} mm/dia" )
    date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
    eval(df = date_pre, onset = True, v = 0, n = 0, step = 10, num = 10)
    ##Correr


Umbral: 2.5 mm/dia
var: 0, lev: 0, bounds: [-110 -100   10   20], corr: 0.476, umbral: -0.500, n. días: 6

Umbral: 3 mm/dia
var: 0, lev: 0, bounds: [-100  -90    0   10], corr: 0.479, umbral: 2.833, n. días: 8

Umbral: 3.5 mm/dia
var: 0, lev: 0, bounds: [-70 -60   0  10], corr: 0.539, umbral: -3.056, n. días: 9


In [10]:
for u in [ 2.5, 3, 3.5 ]:
    print( f"\nUmbral: {u} mm/dia" )
    date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
    eval(df = date_pre, onset = False, v = 0, n = 0, step = 10, num = 10)


Umbral: 2.5 mm/dia
var: 0, lev: 0, bounds: [-60 -50  20  30], corr: 0.519, umbral: 0.222, n. días: 6

Umbral: 3 mm/dia
var: 0, lev: 0, bounds: [-60 -50  20  30], corr: 0.530, umbral: 0.222, n. días: 6

Umbral: 3.5 mm/dia
var: 0, lev: 0, bounds: [-120 -110    0   10], corr: 0.512, umbral: -1.500, n. días: 14


In [ ]:
for u in [ 2.5, 3, 3.5 ]:
    print( f"\nUmbral: {u} mm/dia" )
    date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
    eval(df = date_pre, onset = True, v = 3, n = 0, step = 20, num = 5)

In [ ]:
for u in [ 2.5, 3, 3.5 ]:
    print( f"\nUmbral: {u} mm/dia" )
    date_pre, shift, shift_2 = pre_onset( umbral = [u, u] )
    eval(df = date_pre, onset = False, v = 0, n = 3, step = 20, num = 5)

In [9]:
check( zona = True )


Umbral: 3 mm/dia
var: 0, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.020, umbral: -9.250, n. días: 11
var: 0, lev: 1, bounds: (-80, -70, 12.5, 17.5), corr: 0.037, umbral: 11.250, n. días: 20
var: 1, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.251, umbral: 1.375, n. días: 12
var: 1, lev: 1, bounds: (-80, -70, 12.5, 17.5), corr: 0.161, umbral: -13.500, n. días: 2
var: 2, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.111, umbral: 7760.750, n. días: 15
var: 2, lev: 1, bounds: (-80, -70, 12.5, 17.5), corr: 0.215, umbral: 122110.750, n. días: 13
var: 3, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.182, umbral: 303.000, n. días: 13
var: 4, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.086, umbral: -279.750, n. días: 11
var: 5, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.098, umbral: 101252.250, n. días: 3
var: 6, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.384, umbral: -443.625, n. días: 7
var: 7, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.273, umbral: 117.500, n.

In [10]:
check( onset = False, zona = True)


Umbral: 3 mm/dia
var: 0, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.326, umbral: -9.250, n. días: 20
var: 0, lev: 1, bounds: (-80, -70, 12.5, 17.5), corr: 0.311, umbral: 1.375, n. días: 3
var: 1, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.096, umbral: -2.875, n. días: 1
var: 1, lev: 1, bounds: (-80, -70, 12.5, 17.5), corr: 0.128, umbral: 2.500, n. días: 20
var: 2, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.326, umbral: 7760.750, n. días: 8
var: 2, lev: 1, bounds: (-80, -70, 12.5, 17.5), corr: 0.118, umbral: 121656.500, n. días: 5
var: 3, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.249, umbral: 301.875, n. días: 7
var: 4, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.301, umbral: -279.750, n. días: 1
var: 5, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.109, umbral: 101252.250, n. días: 7
var: 6, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.316, umbral: -443.625, n. días: 5
var: 7, lev: 0, bounds: (-80, -70, 12.5, 17.5), corr: 0.007, umbral: 33.500, n. días: 3

In [ ]:
# var: 0. u, 1. v, 2. gp, 3. sst, 4. olr, 5. sp, 6. vivfu, 7. vivfv
# lev: 0. 925, 1. 200